### Restart and Run All

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3')
conlt = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('display.max_rows',None)

year = 2022
quarter = 1

today = date.today()
today

datetime.date(2022, 5, 4)

In [2]:
sql = '''
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s)
OR (year = %s-1 AND quarter >= %s+1) 
ORDER BY year DESC, quarter DESC'''
sql = sql % (year, quarter, year, quarter)
dfc = pd.read_sql(sql, conlt)
dfc['Counter'] = 1
dfc_grp = dfc.groupby(['name'], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp['Counter'] == 4]
dfc_grp

,name,year,quarter,q_amt,Counter
17,AOT,8085,10,-17151697,4
24,ASP,8085,10,775000,4
31,BAY,8085,10,34707618,4
32,BBL,8085,10,26702007,4
42,BFIT,8085,10,696050,4
45,BH,8085,10,1849587,4
61,CIMBT,8085,10,3160276,4
65,COTTO,8085,10,608126,4
77,DCC,8085,10,1741605,4
80,DELTA,8085,10,7722365,4


In [3]:
sql = '''
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC'''
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2022 AND quarter <= 1-1) 
OR (year = 2022-1 AND quarter >= 1) 
ORDER BY year DESC, quarter DESC


In [4]:
dfp = pd.read_sql(sql, conlt)
dfp['Counter'] = 1
dfp_grp = dfp.groupby(['name'], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp['Counter'] == 4]
dfp_grp.shape

(323, 5)

In [5]:
dfm = pd.merge(dfc_grp, dfp_grp, on='name', suffixes=(['_c','_p']), how='inner')
dfm['inc_profit'] = dfm['q_amt_c']-dfm['q_amt_p']
dfm['Pct'] = round(dfm['inc_profit']/abs(dfm['q_amt_p'])*100,2)
dfm['year'] = year
dfm['quarter'] = 'Q'+str(quarter)
df_pct = dfm[['name','year','quarter','q_amt_c','q_amt_p','inc_profit','Pct']]
df_pct

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
0,AOT,2022,Q1,-17151697,-16322014,-829683,-5.08
1,ASP,2022,Q1,775000,978355,-203355,-20.79
2,BAY,2022,Q1,34707618,33794188,913430,2.70
3,BBL,2022,Q1,26702007,26507039,194968,0.74
4,BFIT,2022,Q1,696050,906775,-210725,-23.24
5,BH,2022,Q1,1849587,1215678,633909,52.14
6,CIMBT,2022,Q1,3160276,2440554,719722,29.49
7,COTTO,2022,Q1,608126,583604,24522,4.20
8,DCC,2022,Q1,1741605,1700429,41176,2.42
9,DELTA,2022,Q1,7722365,6699008,1023357,15.28


In [6]:
sql = """
DELETE FROM qt_profits 
WHERE year = %s AND quarter = 'Q%s'"""
sql = sql % (year,quarter)
print(sql)


DELETE FROM qt_profits 
WHERE year = 2022 AND quarter = 'Q1'


In [7]:
rp = conlt.execute(sql)
rp.rowcount

28

In [8]:
sql = 'SELECT name, id FROM tickers'
tickers = pd.read_sql(sql, conlt)
tickers.shape

(403, 2)

In [9]:
df_ins = pd.merge(df_pct, tickers, on='name', how='inner')
df_ins.dtypes

name           object
year            int64
quarter        object
q_amt_c         int64
q_amt_p         int64
inc_profit      int64
Pct           float64
id              int64
dtype: object

In [10]:
rcds = df_ins.values.tolist()
len(rcds)

30

In [11]:
for rcd in rcds:
    print(rcd)

['AOT', 2022, 'Q1', -17151697, -16322014, -829683, -5.08, 24]
['ASP', 2022, 'Q1', 775000, 978355, -203355, -20.79, 40]
['BAY', 2022, 'Q1', 34707618, 33794188, 913430, 2.7, 49]
['BBL', 2022, 'Q1', 26702007, 26507039, 194968, 0.74, 50]
['BFIT', 2022, 'Q1', 696050, 906775, -210725, -23.24, 58]
['BH', 2022, 'Q1', 1849587, 1215678, 633909, 52.14, 61]
['CIMBT', 2022, 'Q1', 3160276, 2440554, 719722, 29.49, 104]
['COTTO', 2022, 'Q1', 608126, 583604, 24522, 4.2, 710]
['DCC', 2022, 'Q1', 1741605, 1700429, 41176, 2.42, 135]
['DELTA', 2022, 'Q1', 7722365, 6699008, 1023357, 15.28, 138]
['DTAC', 2022, 'Q1', 3260225, 3355933, -95708, -2.85, 144]
['GLOBAL', 2022, 'Q1', 3534362, 3343575, 190787, 5.71, 193]
['GVREIT', 2022, 'Q1', 672205, 705215, -33010, -4.68, 654]
['HMPRO', 2022, 'Q1', 5588994, 5440519, 148475, 2.73, 208]
['IRC', 2022, 'Q1', 245994, 340031, -94037, -27.66, 225]
['KBANK', 2022, 'Q1', 38636812, 38052722, 584090, 1.53, 245]
['KKP', 2022, 'Q1', 6910731, 6318052, 592679, 9.38, 255]
['KSL', 

In [12]:
sql = """
INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) \
 VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)  VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [13]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [14]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit Pct'.split()

In [15]:
criteria_1 = df_ins.q_amt_c > 440000
df_ins.loc[criteria_1,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
5,BH,2022,Q1,1849587,1215678,633909,52.14
22,OISHI,2022,Q1,709629,546690,162939,29.80
6,CIMBT,2022,Q1,3160276,2440554,719722,29.49
9,DELTA,2022,Q1,7722365,6699008,1023357,15.28
18,KTB,2022,Q1,24790196,21588288,3201908,14.83


In [16]:
criteria_2 = df_ins.q_amt_p > 400000
df_ins.loc[criteria_2,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
5,BH,2022,Q1,1849587,1215678,633909,52.14
22,OISHI,2022,Q1,709629,546690,162939,29.80
6,CIMBT,2022,Q1,3160276,2440554,719722,29.49
9,DELTA,2022,Q1,7722365,6699008,1023357,15.28
18,KTB,2022,Q1,24790196,21588288,3201908,14.83


In [17]:
criteria_3 = df_ins.Pct > 10
df_ins.loc[criteria_3,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
5,BH,2022,Q1,1849587,1215678,633909,52.14
22,OISHI,2022,Q1,709629,546690,162939,29.80
6,CIMBT,2022,Q1,3160276,2440554,719722,29.49
9,DELTA,2022,Q1,7722365,6699008,1023357,15.28
18,KTB,2022,Q1,24790196,21588288,3201908,14.83


In [18]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
5,BH,2022,Q1,1849587,1215678,633909,52.14
22,OISHI,2022,Q1,709629,546690,162939,29.80
6,CIMBT,2022,Q1,3160276,2440554,719722,29.49
9,DELTA,2022,Q1,7722365,6699008,1023357,15.28
18,KTB,2022,Q1,24790196,21588288,3201908,14.83
